In [ ]:
import sys
import json
import math


def calculate_reldcg(results):
    return sum(doc['relevance'] / (i + 1) for i, doc in enumerate(results))


def calculate_revenue(results):
    return sum(doc['cost'] / math.sqrt(i + 1) for i, doc in enumerate(results))


def optimize_serp(original_results, new_docs):
    original_reldcg = calculate_reldcg(original_results)
    original_revenue = calculate_revenue(original_results)
    best_results = original_results.copy()
    max_revenue = original_revenue

    for new_doc in new_docs:
        for insert_pos in range(len(original_results) + 1):
            new_list = original_results.copy()
            new_list.insert(insert_pos, new_doc)
            if len(new_list) > len(original_results):
                new_list = new_list[:len(original_results)]
            
            current_reldcg = calculate_reldcg(new_list)
            if current_reldcg < original_reldcg:
                continue
            
            current_revenue = calculate_revenue(new_list)
            if current_revenue > max_revenue:
                max_revenue = current_revenue
                best_results = new_list.copy()

    return best_results


def process_input(input_data):
    query_to_new_docs = {}
    for doc in input_data['new_documents']:
        query = doc['query']
        if query not in query_to_new_docs:
            query_to_new_docs[query] = []
        query_to_new_docs[query].append(doc)

    total_revenue = 0.0
    for serp in input_data['serpset']:
        query = serp['query']
        original_results = sorted(serp['results'], key=lambda x: x['position'])
        new_docs = query_to_new_docs.get(query, [])
        
        optimized_results = optimize_serp(original_results, new_docs)
        total_revenue += calculate_revenue(optimized_results)
    
    return round(total_revenue, 2)


def main():
    with open('input.txt', 'r') as f:
        input_data = json.load(f)
    
    total_revenue = process_input(input_data)
    print(f"{total_revenue:.2f}")


if __name__ == '__main__':
    main()